# Import

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from datetime import datetime
from datetime import date
import time
import os
import seaborn as sns

# Cào dữ liệu

- Dữ liệu được cào từ trang web https://www.centralcharts.com/en/price-list-ranking/ALL/asc/ts_19-us-nasdaq-stocks--qc_1-alphabetical-order?p=1 của sàn chứng khoán US NASDAQ Stocks price list and quotes.

- Dữ liệu được cào hằng ngày, vào khoảng sau 21h00. Điều này sẽ phục vụ cho mô hình dự đoán giá cổ phiếu theo ngày.

In [2]:
pages = []
# There are 155 pages in the entire set.
for page_number in range(1, 155):
    url = 'https://www.centralcharts.com/en/price-list-ranking/ALL/asc/ts_19-us-nasdaq-stocks--qc_1-alphabetical-order?p='
    url = url + str(page_number)
    pages.append(url)

# Create a list containing the <th> tag contents.
webpage = requests.get(pages[0])
soup = bs(webpage.text, 'html.parser')
stock_table = soup.find('table', class_='tabMini tabQuotes')
th_tag_list = stock_table.find_all('th')

# Take the <th> list and remove the extra attributes to
# get just the text portion of each tag. Use this text for the
# column header labels in the dataframe.
headers = []
for each_tag in th_tag_list:
    title = each_tag.text
    headers.append(title)

headers[0] = 'Name'

new_headers = []
for header in headers:
    if header not in ('Cap.', 'Issued Cap.', ''):
        new_headers.append(header)
headers = new_headers
stock_df = pd.DataFrame(columns = headers)

# Cycle through each page.
for page in pages:
    webpage = requests.get(page, timeout=60)
    soup = bs(webpage.text, 'html.parser')

    # Check to see if the page contains a table. If it does,
    # create a list of <tr> tags. If not, go to the next page.
    if soup.find('table'):
        stock_table = soup.find('table', class_='tabMini tabQuotes')
        tr_tag_list = stock_table.find_all('tr')

        # Cycle through the <tr> list. For each
        # row, find the <td> tags within the row. Then
        # obtain the text within each <td> tag. Lastly,
        # place the text in the last row of the dataframe.
        for each_tr_tag in tr_tag_list[1:]:
            td_tag_list = each_tr_tag.find_all('td')

            row_values = []
            for each_td_tag in td_tag_list[0:7]:
                new_value = each_td_tag.text.strip()
                row_values.append(new_value)

            stock_df.loc[len(stock_df)] = row_values

Sau khi cào dữ liệu về, ta đánh dấu cột `date` chính là ngày cào dữ liệu. Mỗi lần cào tương ứng với ngày hôm đó.

In [3]:
stock_df['date'] = date.today().strftime('%m/%d/%Y')
stock_df

,Name,Current price,Change(%),Open,High,Low,Volume,date
0,1-800-FLOWERS.COM INC.,5.08,-1.55%,5.21,5.21,5.08,"26,840",06/11/2025
1,10X GENOMICS INC.,10.97,+3.00%,10.85,11.12,10.71,"472,770",06/11/2025
2,111 INC. ADS,8.21,+1.73%,8.21,8.21,8.21,100,06/11/2025
3,17 EDUCATION & TECHNOLOGY GROUP,1.96,-7.11%,2.07,2.07,1.96,"2,554",06/11/2025
4,180 LIFE SCIENCES,1.04,+0.00%,1.03,1.04,1.03,"3,646",06/11/2025
...,...,...,...,...,...,...,...,...
3607,ZURA BIO LTD.,1.30,+1.56%,1.29,1.33,1.25,"66,410",06/11/2025
3608,ZW DATA ACTION TECHNOLOGIES,1.46,+15.87%,1.61,1.64,1.46,"521,670",06/11/2025
3609,ZYMEWORKS INC.,13.09,+0.15%,13.19,13.22,13.01,"19,482",06/11/2025
3610,ZYNEX INC.,2.14,-2.28%,2.18,2.22,2.14,"5,134",06/11/2025


Lưu dữ liệu đã cào cho 1 ngày vào thư mục log.

In [4]:
file_name = 'stock_data-' + date.today().strftime('%Y-%m-%d') + '.csv'
if not os.path.exists('log'):
    os.makedirs('log')
stock_df.to_csv('log/' + file_name, index=False)
# Export the dataframe to a csv file.

Đọc file tổng hợp và xem thử cho tới trước ngày hôm nay, chúng ta đã cào được bao nhiêu.

In [6]:
integrated_df = pd.read_csv('integrated_stock_data.csv')
integrated_df

,Name,Current price,Change(%),Open,High,Low,Volume,date
0,MGO GLOBAL INC.,0.37,0,0.399,0.4599,0.33,111379653,3/5/2025
1,VISIONARY HOLDINGS INC.,2.51,94.57,2.28,2.85,2.17,24468952,3/5/2025
2,ADITXT INC.,0.0632,26.91,0.0635,0.067,0.0584,24278165,3/5/2025
3,NVIDIA CORP.,116.75,0.66,117.58,118,115.04,19755191,3/5/2025
4,MATTERPORT INC.,5.38,0,5.42,5.45,5.35,13993565,3/5/2025
...,...,...,...,...,...,...,...,...
351496,ZURA BIO LTD.,1.3,4.00%,1.27,1.37,1.25,"275,758",6/10/2025
351497,ZW DATA ACTION TECHNOLOGIES,1.25,-0.79%,1.26,1.26,1.25,"2,528",6/10/2025
351498,ZYMEWORKS INC.,12.73,2.25%,12.54,12.82,12.45,"39,483",6/10/2025
351499,ZYNEX INC.,2.2,0.46%,2.21,2.25,2.2,"24,144",6/10/2025


Sau đó, tích hợp lượng dự liệu đã cào hôm nay vào file tổng hợp. File này sẽ dùng cho phân tích, dự đoán giá cổ phiếu cho tương lai, dựa trên giá trị cổ phiếu của các ngày qua.

In [7]:
integrated_df = pd.concat([integrated_df, stock_df], ignore_index=True)
integrated_df.to_csv('integrated_stock_data.csv', index=False)
integrated_df

,Name,Current price,Change(%),Open,High,Low,Volume,date
0,MGO GLOBAL INC.,0.37,0,0.399,0.4599,0.33,111379653,3/5/2025
1,VISIONARY HOLDINGS INC.,2.51,94.57,2.28,2.85,2.17,24468952,3/5/2025
2,ADITXT INC.,0.0632,26.91,0.0635,0.067,0.0584,24278165,3/5/2025
3,NVIDIA CORP.,116.75,0.66,117.58,118,115.04,19755191,3/5/2025
4,MATTERPORT INC.,5.38,0,5.42,5.45,5.35,13993565,3/5/2025
...,...,...,...,...,...,...,...,...
355108,ZURA BIO LTD.,1.30,+1.56%,1.29,1.33,1.25,"66,410",06/11/2025
355109,ZW DATA ACTION TECHNOLOGIES,1.46,+15.87%,1.61,1.64,1.46,"521,670",06/11/2025
355110,ZYMEWORKS INC.,13.09,+0.15%,13.19,13.22,13.01,"19,482",06/11/2025
355111,ZYNEX INC.,2.14,-2.28%,2.18,2.22,2.14,"5,134",06/11/2025


# Khám phá dữ liệu

#### Đọc dữ liệu và hiển thị data frame

In [8]:
df = pd.read_csv('integrated_stock_data.csv')
df

,Name,Current price,Change(%),Open,High,Low,Volume,date
0,MGO GLOBAL INC.,0.37,0,0.399,0.4599,0.33,111379653,3/5/2025
1,VISIONARY HOLDINGS INC.,2.51,94.57,2.28,2.85,2.17,24468952,3/5/2025
2,ADITXT INC.,0.0632,26.91,0.0635,0.067,0.0584,24278165,3/5/2025
3,NVIDIA CORP.,116.75,0.66,117.58,118,115.04,19755191,3/5/2025
4,MATTERPORT INC.,5.38,0,5.42,5.45,5.35,13993565,3/5/2025
...,...,...,...,...,...,...,...,...
355108,ZURA BIO LTD.,1.30,+1.56%,1.29,1.33,1.25,"66,410",06/11/2025
355109,ZW DATA ACTION TECHNOLOGIES,1.46,+15.87%,1.61,1.64,1.46,"521,670",06/11/2025
355110,ZYMEWORKS INC.,13.09,+0.15%,13.19,13.22,13.01,"19,482",06/11/2025
355111,ZYNEX INC.,2.14,-2.28%,2.18,2.22,2.14,"5,134",06/11/2025


#### Kiểm tra các ngày ta đã ghi nhận giá cổ phiếu

In [9]:
unique_dates = df['date'].unique()
unique_dates

array(['3/5/2025', '3/6/2025', '3/7/2025', '3/8/2025', '3/9/2025',
       '3/10/2025', '3/11/2025', '3/12/2025', '3/13/2025', '3/14/2025',
       '3/15/2025', '3/16/2025', '3/17/2025', '3/18/2025', '3/19/2025',
       '3/20/2025', '3/21/2025', '3/22/2025', '3/23/2025', '3/24/2025',
       '3/25/2025', '3/26/2025', '3/27/2025', '3/28/2025', '3/30/2025',
       '3/31/2025', '3/29/2025', '4/1/2025', '4/2/2025', '4/3/2025',
       '4/4/2025', '4/5/2025', '4/6/2025', '4/7/2025', '4/8/2025',
       '4/9/2025', '4/10/2025', '4/11/2025', '4/12/2025', '4/13/2025',
       '4/14/2025', '4/15/2025', '4/16/2025', '4/17/2025', '4/18/2025',
       '4/19/2025', '4/20/2025', '4/21/2025', '4/22/2025', '4/23/2025',
       '4/24/2025', '4/25/2025', '4/26/2025', '4/27/2025', '4/28/2025',
       '4/29/2025', '4/30/2025', '5/1/2025', '5/2/2025', '5/3/2025',
       '5/4/2025', '5/5/2025', '5/6/2025', '5/7/2025', '5/8/2025',
       '5/9/2025', '5/10/2025', '5/11/2025', '5/12/2025', '5/13/2025',
       '5/14/20

#### Xác định số hàng và số cột

In [10]:
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 355113, Columns: 8


#### Xác định thuộc tính trong tệp dữ liệu

In [11]:
df.columns.tolist()

['Name', 'Current price', 'Change(%)', 'Open', 'High', 'Low', 'Volume', 'date']

🧾 1. Name
- **Ý nghĩa:** Tên của công ty hoặc mã cổ phiếu.  
- **Ví dụ:** `NVIDIA CORP.`, `TESLA INC.`, `ADITXT INC.`  
- **Công dụng:** Xác định đơn vị cổ phiếu được theo dõi.


💵 2. Current price
- **Ý nghĩa:** Giá cổ phiếu hiện tại tại thời điểm ghi nhận (ngày 3/5/2025).  
- **Đơn vị:** USD (đô la Mỹ).  
- **Công dụng:** Giá đóng cửa cuối cùng trong ngày, là cơ sở để tính lời/lỗ.


📈 3. Change(%)
- **Ý nghĩa:** Tỷ lệ phần trăm thay đổi của giá cổ phiếu so với phiên trước đó.  
- **Cách tính:**  
$$
\text{Change(\%)} = \left( \frac{\text{Current Price} - \text{Previous Close}}{\text{Previous Close}} \right) \times 100
$$


- **Công dụng:** Đánh giá mức tăng/giảm trong ngày.


🔓 4. Open
- **Ý nghĩa:** Giá mở cửa – giá đầu tiên khi cổ phiếu bắt đầu giao dịch trong ngày.  
- **Công dụng:** So sánh với giá hiện tại để biết xu hướng giá trong ngày.


🔺 5. High
- **Ý nghĩa:** Mức giá cao nhất mà cổ phiếu đạt được trong ngày.  
- **Công dụng:** Cho thấy biên độ biến động của cổ phiếu.


🔻 6. Low
- **Ý nghĩa:** Mức giá thấp nhất trong ngày.  
- **Công dụng:** Kết hợp với “High” để đánh giá độ biến động giá.


📊 7. Volume
- **Ý nghĩa:** Tổng số cổ phiếu đã được giao dịch trong ngày.  
- **Công dụng:** Đánh giá tính thanh khoản.  
  - **Cao:** Nhiều người mua/bán → cổ phiếu "nóng" hoặc phổ biến.  
  - **Thấp:** Ít người quan tâm hoặc đang giữ dài hạn.


📅 8. Date
- **Ý nghĩa:** Ngày dữ liệu được ghi nhận.  
- **Trong dataset:** Từ ngày 5/3/2025 đến ngày 4/5/2025  
- **Công dụng:** Nếu có nhiều ngày, sẽ dùng để phân tích xu hướng theo thời gian.


#### Xác định kiểu dữ liệu cho từng thuộc tính

In [12]:
df.dtypes

Name             object
Current price    object
Change(%)        object
Open             object
High             object
Low              object
Volume           object
date             object
dtype: object

#### Xóa bỏ cột không cần thiết như `Change(%)`

In [13]:
df = df.drop(columns=['Change(%)'])

#### Đổi sang kiểu dữ liệu phù hợp

In [14]:
numeric_cols = ['Current price', 'Open', 'High', 'Low', 'Volume']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace('[^0-9.-]', '', regex=True), errors='coerce')

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [15]:
df.dtypes

Name                     object
Current price           float64
Open                    float64
High                    float64
Low                     float64
Volume                  float64
date             datetime64[ns]
dtype: object

## Tiền xử lý dữ liệu

#### Dữ liệu trùng lặp

In [16]:
df.duplicated().sum()

132

Xoá dữ liệu trùng lặp

In [17]:
df = df.drop_duplicates()
df.shape

(354981, 7)

#### Xử lý dữ liệu bị thiếu

In [18]:
df = df.dropna()

## Tiền xử lý dữ liệu cho modeling

Bởi vì mỗi loại cổ phiếu tồn tại bền vững đều phải có đúng 1 giá trị ở 1 ngày. Nên nếu cổ phiếu nào không khớp với số ngày thì phải loại bỏ.

Trước tiên, tính số ngày mà thu thập cổ phiếu cho tới hôm nay.

In [19]:
num_days = df['date'].nunique()
num_days

99

Như vậy, cổ phiếu nào mà số ngày xuất hiện không đủ như trên, thì loại bỏ.

In [20]:
name_counts = df['Name'].value_counts()
valid_names = name_counts[name_counts == num_days].index

# Giữ lại các dòng có 'Name' nằm trong danh sách đầy đủ số ngày
df = df[df['Name'].isin(valid_names)]
df.shape

(280764, 7)

Lưu vào file dữ liệu đã làm sạch

In [21]:
df.to_csv('cleaned_integrated_stock_data.csv', index=False)